## POS tagging using HMMs - Lab 9

In [2]:
#All imports
import nltk
from nltk.corpus import brown
from sklearn.model_selection import KFold
import numpy as np

# Download the Brown corpus and the universal tagset
nltk.download("brown")
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [3]:
# Get the tagged sentences from the Brown corpus
tagged_sentences = brown.tagged_sents(tagset="universal")
tagged_sentences = list(tagged_sentences)

In [4]:
tagged_sentences

[[('The', 'DET'),
  ('Fulton', 'NOUN'),
  ('County', 'NOUN'),
  ('Grand', 'ADJ'),
  ('Jury', 'NOUN'),
  ('said', 'VERB'),
  ('Friday', 'NOUN'),
  ('an', 'DET'),
  ('investigation', 'NOUN'),
  ('of', 'ADP'),
  ("Atlanta's", 'NOUN'),
  ('recent', 'ADJ'),
  ('primary', 'NOUN'),
  ('election', 'NOUN'),
  ('produced', 'VERB'),
  ('``', '.'),
  ('no', 'DET'),
  ('evidence', 'NOUN'),
  ("''", '.'),
  ('that', 'ADP'),
  ('any', 'DET'),
  ('irregularities', 'NOUN'),
  ('took', 'VERB'),
  ('place', 'NOUN'),
  ('.', '.')],
 [('The', 'DET'),
  ('jury', 'NOUN'),
  ('further', 'ADV'),
  ('said', 'VERB'),
  ('in', 'ADP'),
  ('term-end', 'NOUN'),
  ('presentments', 'NOUN'),
  ('that', 'ADP'),
  ('the', 'DET'),
  ('City', 'NOUN'),
  ('Executive', 'ADJ'),
  ('Committee', 'NOUN'),
  (',', '.'),
  ('which', 'DET'),
  ('had', 'VERB'),
  ('over-all', 'ADJ'),
  ('charge', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('election', 'NOUN'),
  (',', '.'),
  ('``', '.'),
  ('deserves', 'VERB'),
  ('the', 'DET'),

Creating Mapping from States and Observation Traces to numbers

In [5]:
states=[]
observations=[]

for j in tagged_sentences:
    for i in j:
        if not(i[-1] in states):
            states.append(i[-1])
        if not(i[0] in observations):
            observations.append(i[0])

print(len(states),len(observations))

12 56057


In [38]:
wholeData=[]
for j in tagged_sentences:
    for i in j:
        wholeData.append(i)

In [6]:
#mapping from states to numbers (indices in matrix)
stateMap={'BOS':0}
j=1
for i in states:
    stateMap[i]=j
    j=j+1
stateMap['EOS']=j
print(stateMap)

#mapping from observation traces to numbers
obsMap={'eps':0}
k=1
for i in observations:
    obsMap[i]=k
    k=k+1
        

{'BOS': 0, 'DET': 1, 'NOUN': 2, 'ADJ': 3, 'VERB': 4, 'ADP': 5, '.': 6, 'ADV': 7, 'CONJ': 8, 'PRT': 9, 'PRON': 10, 'NUM': 11, 'X': 12, 'EOS': 13}


In [7]:
#using KFold from sklearn to create folds
kf=KFold(n_splits=5,shuffle=False)

OneSmoothedTrans=[]
OneSmoothedEmm=[]

for trainIndices,testIndices in kf.split(tagged_sentences):

    training=[tagged_sentences[i] for i in trainIndices]
    test=[tagged_sentences[i] for i in testIndices]

    obsStateMatrix=np.zeros((len(states)+2,len(observations)+1))+1
    stateMatrix=np.zeros((len(states)+2,len(states)+2))+1
    y=len(tagged_sentences)
    for i in range(y):
        z=len(tagged_sentences[i])
        for j in range(z):
            if j==0:
                stateMatrix[stateMap['BOS']][stateMap[tagged_sentences[i][j][-1]]]=stateMatrix[stateMap['BOS']][stateMap[tagged_sentences[i][j][-1]]]+1
                stateMatrix[stateMap['BOS']][obsMap['eps']]=stateMatrix[stateMap['BOS']][obsMap['eps']]+1
            if j==z-1:
                stateMatrix[stateMap['EOS']][stateMap['BOS']]=stateMatrix[stateMap['EOS']][stateMap['BOS']]+1
                stateMatrix[stateMap['EOS']][obsMap['eps']]=stateMatrix[stateMap['EOS']][obsMap['eps']]+1
            if j<z-1:
                nextState=tagged_sentences[i][j+1][-1]
                currObs=tagged_sentences[i][j][0]
                currState=tagged_sentences[i][j][-1]

                stateMatrix[stateMap[currState]][stateMap[nextState]]=stateMatrix[stateMap[currState]][stateMap[nextState]]+1
                obsStateMatrix[stateMap[currState]][obsMap[currObs]]=obsStateMatrix[stateMap[currState]][obsMap[currObs]]+1
                
    stateMatrix=stateMatrix/(stateMatrix.sum(axis=1,keepdims=True))
    obsStateMatrix=obsStateMatrix/(obsStateMatrix.sum(axis=1,keepdims=True))
    OneSmoothedEmm.append(obsStateMatrix)
    OneSmoothedTrans.append(stateMatrix)
                

<h3>Split 1</h3>

In [58]:
print("transition matrix")
print(OneSmoothedTrans[0])
print()
print("Emission Matrix")
print(OneSmoothedEmm[0])

transition matrix
[[4.99947687e-01 1.06710028e-01 7.05703873e-02 1.71761382e-02
  2.25731076e-02 6.14243116e-02 4.44661447e-02 4.56780651e-02
  2.45697247e-02 1.83444644e-02 7.98472457e-02 8.41369209e-03
  2.70284409e-04 8.71885190e-06]
 [7.29847097e-06 5.91176149e-03 6.26493450e-01 2.39732876e-01
  6.46790497e-02 9.07929789e-03 1.27869211e-02 1.75236288e-02
  6.49563916e-04 2.01437799e-03 9.91862205e-03 9.77265263e-03
  1.42320184e-03 7.29847097e-06]
 [3.64089158e-06 1.55502479e-02 1.49804484e-01 1.29397287e-02
  1.59339979e-01 2.45126667e-01 2.84652186e-01 2.64802045e-02
  5.98853847e-02 1.78986230e-02 1.98829089e-02 8.09370199e-03
  3.38602917e-04 3.64089158e-06]
 [1.19468604e-05 5.85396158e-03 6.52848132e-01 5.69387365e-02
  1.74902036e-02 8.85023416e-02 1.00413361e-01 9.68890376e-03
  3.76087164e-02 1.92941795e-02 3.83494218e-03 6.98891331e-03
  5.13714996e-04 1.19468604e-05]
 [5.47459242e-06 1.63060735e-01 9.75572369e-02 5.75434409e-02
  1.84318577e-01 1.69307245e-01 8.06626447e-

In [ ]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
        for y in states:
            (prob, state) = max(
                [(V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states]
            )
            V[t][y] = prob
            newpath[y] = path[state] + [y]
        path = newpath
    (prob, state) = max([(V[-1][y], y) for y in states])
    return (prob, path[state])

# states = ('S0', 'S1')
# observations = ('A1', 'A2', 'A3')
# start_probability = {'S0': 0.6, 'S1': 0.4}
# transition_probability = {
#    'S0' : {'S0': 0.7, 'S1': 0.3},
#    'S1' : {'S0': 0.4, 'S1': 0.6},
#    }
# emission_probability = {
#    'S0' : {'A1': 0.1, 'A2': 0.4, 'A3': 0.5},
#    'S1' : {'A1': 0.6, 'A2': 0.3, 'A3': 0.1},
#    }

print(viterbi(observations, states, start_probability, OneSmoothedTrans[0], OneSmoothedEmm[0]))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices